In [ ]:
import uproot as up
import matplotlib.pyplot as plt
import numpy as np
import awkward as ak
import re

from uproot3_methods.classes.TLorentzVector import TLorentzVector
mag2 = TLorentzVector.mag2

In [ ]:
file = up.open("../build/evtOutput.root")

In [ ]:
class rootDF:
    def __init__(self, file_name):
        file = up.open(file_name)
        ntp = file["ntp"]
        self.Px = ntp["px"].array()
        self.Py = ntp["py"].array()
        self.Pz = ntp["pz"].array()
        self.E = ntp["E"].array()
        self.nTrk = ntp["nTrk"].array()
        file.close()
        self.filter = np.array([True]*len(self.Px))
    def sign(self, x):
        return -1 if x<0 else 1
    def get_p_from_list(self, iEv, part_list):
        p = TLorentzVector(0, 0, 0, 0)
        for i in part_list:
            if( max(np.abs(part_list)) >= self.nTrk[iEv]):
                continue
            abs_i = abs(i)
            p += self.sign(i)*TLorentzVector(
                self.Px[iEv][abs_i], self.Py[iEv][abs_i], self.Pz[iEv][abs_i], self.E[iEv][abs_i]
            )
        return p
    def size(self):
        return len(self.Px)
    def apply(self, func, iEv, part_list):
        p = self.get_p_from_list(iEv, part_list)
        try:
            return func.fget(p)
        except ValueError:
            return 0
    def m2(self, iEv, part_list):
        return self.apply(TLorentzVector.mag2, iEv, part_list)
    def m(self, iEv, part_list):
        return self.apply(TLorentzVector.mag, iEv, part_list)
    def pt2(self, iEv, part_list):
        return self.apply(TLorentzVector.pt2, iEv, part_list)
    def pt(self, iEv, part_list):
        return self.apply(TLorentzVector.pt, iEv, part_list)
    def parse_var(self, var):
        [name, numbers] = var.split("_")
        func = {'m2':TLorentzVector.mag2,
                'm': TLorentzVector.mag,
                'pt2': TLorentzVector.pt2,
                 'pt': TLorentzVector.pt}[name]
        part_list = [int(n) for n in re.sub('([0-9])',r'\1 ',numbers.replace('m','-')).split(' ')[:-1]]
        return func, part_list
    
    def calc_var(self, iEv, var):
        func, part_list = self.parse_var(var)
        return self.apply(func, iEv, part_list)
    def __getitem__(self, key):
        if type(key)==str:
            func, part_list = self.parse_var(key)
            return np.array([self.apply(func, iEv, part_list) for iEv in range(self.size()) if self.filter[iEv]])
        if type(key) == list:
            return [self[k] for k in key]
    def cut(self, condition):
        if (condition == True):
            self.filter = [True]*self.size()
        elif type(condition) == str:
            condition = re.sub('([<>])',r' \1 ', condition)
            var, operation, lhs = [s for s in condition.split(' ') if len(s)>0]
            lhs = float(lhs)
            if(operation == '>'):
                self.filter = [self.filter[iEv] & (self.calc_var(iEv, var) > lhs) for iEv in range( self.size())]
            elif(operation == '<'):
                self.filter = [self.filter[iEv] & (self.calc_var(iEv, var) < lhs) for iEv in range( self.size())]
        elif type(condition) == list:
            for cond in condition:
                self.cut(cond)
        return np.count_nonzero(self.filter)

In [ ]:
df = rootDF('../build/evtOutput.root')

In [ ]:
df.cut(['m_12>1', 'm_13 < 0.5'])

In [ ]:
plt.hist( df["m_12"], bins = 100);